In [1]:
import muon as mu
import scvi
import os
import anndata as ad
import scanpy as sc
import numpy as np

/home/ceger/miniforge3/envs/scvi_muon-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
data_path = os.path.join(os.getcwd(), '.data')
input_path = os.path.join(data_path, 'input')
output_path = os.path.join(data_path, 'output')

os.listdir(input_path)

['E-MTAB12916_E-MTAB12919-QC-Norm.h5mu']

In [3]:
mdata = mu.read_h5mu(os.path.join(input_path, 'E-MTAB12916_E-MTAB12919-QC-Norm.h5mu'))
mdata

MuData object with n_obs × n_vars = 213373 × 3775615
  var:	'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
  2 modalities
    rna:	213373 x 70711
      obs:	'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes', 'doublet_score', 'predicted_doublet'
      var:	'gene_ids', 'feature_types', 'genome', 'interval', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
      uns:	'scrublet'
      layers:	'log1p_norm'
    atac:	213373 x 3704904
      obs:	'sample', 'n_features_per_cell', 'total_fragment_counts', 'log_total_fragment_counts'
      var:	'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
      uns:	'atac', 'files'
      layers:	'log1p_norm'

In [4]:
rna = mdata.mod['rna']
rna

AnnData object with n_obs × n_vars = 213373 × 70711
    obs: 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes', 'doublet_score', 'predicted_doublet'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'scrublet'
    layers: 'log1p_norm'

In [5]:
num_cells_to_keep = 80000
np.random.seed(42)  # for reproducibility
cells_to_keep = np.random.choice(rna.obs_names, size=num_cells_to_keep, replace=False)
num_genes_to_keep = 20000
genes_to_keep = np.random.choice(rna.var_names, size=num_genes_to_keep, replace=False)
rna = rna[cells_to_keep, genes_to_keep]
rna

View of AnnData object with n_obs × n_vars = 80000 × 20000
    obs: 'sample', 'n_genes_by_counts', 'log1p_n_genes_by_counts', 'total_counts', 'log1p_total_counts', 'pct_counts_in_top_20_genes', 'total_counts_mt', 'log1p_total_counts_mt', 'pct_counts_mt', 'total_counts_ribo', 'log1p_total_counts_ribo', 'pct_counts_ribo', 'total_counts_hb', 'log1p_total_counts_hb', 'pct_counts_hb', 'outlier', 'mt_outlier', 'n_genes', 'doublet_score', 'predicted_doublet'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'mt', 'ribo', 'hb', 'n_cells_by_counts', 'mean_counts', 'log1p_mean_counts', 'pct_dropout_by_counts', 'total_counts', 'log1p_total_counts'
    uns: 'scrublet'
    layers: 'log1p_norm'

In [6]:
atac = mdata.mod['atac']
atac

AnnData object with n_obs × n_vars = 213373 × 3704904
    obs: 'sample', 'n_features_per_cell', 'total_fragment_counts', 'log_total_fragment_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'atac', 'files'
    layers: 'log1p_norm'

In [7]:
nump_peaks_to_keep = 500000
peaks_to_keep = np.random.choice(atac.var_names, size=nump_peaks_to_keep, replace=False)
atac = atac[cells_to_keep, peaks_to_keep]
atac

View of AnnData object with n_obs × n_vars = 80000 × 500000
    obs: 'sample', 'n_features_per_cell', 'total_fragment_counts', 'log_total_fragment_counts'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    uns: 'atac', 'files'
    layers: 'log1p_norm'

In [8]:
n_genes = len(rna.var_names)
n_regions = len(atac.var_names)

In [9]:
adata_paired = ad.concat([rna.copy().T, atac.copy().T]).T
adata_paired.obs = adata_paired.obs.join(rna.obs[["sample"]])
adata_paired.obs["modality"] = "paired"
adata_paired

AnnData object with n_obs × n_vars = 80000 × 520000
    obs: 'sample', 'modality'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    layers: 'log1p_norm'

In [10]:
adata_mvi = scvi.data.organize_multiome_anndatas(adata_paired)
adata_mvi

AnnData object with n_obs × n_vars = 80000 × 520000
    obs: 'sample', 'modality'
    var: 'gene_ids', 'feature_types', 'genome', 'interval', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts'
    layers: 'log1p_norm'

In [11]:
scvi.model.MULTIVI.setup_anndata(
    adata_mvi,
    batch_key="modality",
    categorical_covariate_keys=["sample"],
)

An NVIDIA GPU may be present on this machine, but a CUDA-enabled jaxlib is not installed. Falling back to cpu.
/home/ceger/miniforge3/envs/scvi_muon-env/lib/python3.10/site-packages/scvi/data/fields/_layer_field.py:116: UserWarning: Training will be faster when sparse matrix is formatted as CSR. It is safe to cast before model initialization.
  _verify_and_correct_data_format(adata, self.attr_name, self.attr_key)


In [12]:
model = scvi.model.MULTIVI(
    adata_mvi,
    n_genes=(adata_mvi.var["feature_types"] == "Gene Expression").sum(),
    n_regions=(adata_mvi.var["feature_types"] == "Peaks").sum(),
)
model.view_anndata_setup()

/home/ceger/miniforge3/envs/scvi_muon-env/lib/python3.10/site-packages/torch/nn/init.py:452: UserWarning: Initializing zero-element tensors is a no-op
  warnings.warn("Initializing zero-element tensors is a no-op")


Anndata setup with scvi-tools version 1.1.3.

Setup via `MULTIVI.setup_anndata` with arguments:

{
│   'layer': None,
│   'batch_key': 'modality',
│   'size_factor_key': None,
│   'categorical_covariate_keys': ['sample'],
│   'continuous_covariate_keys': None,
│   'protein_expression_obsm_key': None,
│   'protein_names_uns_key': None
}

         Summary Statistics          
┏━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃     Summary Stat Key     ┃ Value  ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│         n_batch          │   1    │
│         n_cells          │ 80000  │
│ n_extra_categorical_covs │   1    │
│ n_extra_continuous_covs  │   0    │
│         n_labels         │   1    │
│          n_vars          │ 520000 │
└──────────────────────────┴────────┘

                             Data Registry                             
┏━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃      Registry Key      ┃            scvi-tools Location             ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│           X            │                  adata.X                   │
│         batch          │          adata.obs['_scvi_batch']          │
│ extra_categorical_covs │ adata.obsm['_scvi_extra_categorical_covs'] │
│         ind_x          │           adata.obs['_indices']            │
│         labels         │         adata.obs['_scvi_labels']          │
└────────────────────────┴────────────────────────────────────────────┘

                    batch State Registry                    
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃    Source Location    ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['modality'] │   paired   │          0          │
└───────────────────────┴────────────┴─────────────────────┘

                     labels State Registry                      
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃      Source Location      ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['_scvi_labels'] │     0      │          0          │
└───────────────────────────┴────────────┴─────────────────────┘

                    batch State Registry                    
┏━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃    Source Location    ┃ Categories ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['modality'] │   paired   │          0          │
└───────────────────────┴────────────┴─────────────────────┘

                        extra_categorical_covs State Registry                        
┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━┓
┃   Source Location   ┃              Categories               ┃ scvi-tools Encoding ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━┩
│ adata.obs['sample'] │    HCAHeart9508627_HCAHeart9508819    │          0          │
│                     │    HCAHeart9508628_HCAHeart9508820    │          1          │
│                     │    HCAHeart9508629_HCAHeart9508821    │          2          │
│                     │    HCAHeart9845431_HCAHeart9917173    │          3          │
│                     │    HCAHeart9845432_HCAHeart9917174    │          4          │
│                     │    HCAHeart9845433_HCAHeart9917175    │          5          │
│                     │    HCAHeart9845434_HCAHeart9917176    │          6          │
│                     │    HCAHeart9845435_HCAHeart9917177    │          7          │
│                     │    HCAHeart9845436_HCAHeart9917178    │          8          │
│                     │ HCAHeartST10773165_HCAHeartST10781062 │          9          │
│                     │ HCAHeartST10773166_HCAHeartST10781063 │         10          │
│                     │ HCAHeartST10773167_HCAHeartST10781064 │         11          │
│                     │ HCAHeartST10773168_HCAHeartST10781065 │         12          │
│                     │ HCAHeartST10773169_HCAHeartST10781446 │         13          │
│                     │ HCAHeartST10773170_HCAHeartST10781447 │         14          │
│                     │ HCAHeartST10773171_HCAHeartST10781448 │         15          │
│                     │ HCAHeartST11064574_HCAHeartST11023239 │         16          │
│                     │ HCAHeartST11064575_HCAHeartST11023240 │         17          │
│                     │ HCAHeartST11064576_HCAHeartST11023241 │         18          │
│                     │ HCAHeartST11064577_HCAHeartST11023242 │         19          │
│                     │ HCAHeartST11350184_HCAHeartST11121842 │         20          │
│                     │ HCAHeartST11350186_HCAHeartST11121844 │         21          │
│                     │ HCAHeartST11350187_HCAHeartST11121845 │         22          │
│                     │ HCAHeartST11350192_HCAHeartST11445769 │         23          │
│                     │ HCAHeartST11350193_HCAHeartST11445770 │         24          │
│                     │ HCAHeartST11350195_HCAHeartST11445772 │         25          │
│                     │ HCAHeartST11350196_HCAHeartST11445773 │         26          │
│                     │ HCAHeartST11350197_HCAHeartST11445774 │         27          │
│                     │ HCAHeartST11350198_HCAHeartST11445775 │         28          │
│                     │ HCAHeartST13146201_HCAHeartST13129134 │         29          │
│                     │ HCAHeartST13146202_HCAHeartST13129135 │         30          │
│                     │ HCAHeartST13146203_HCAHeartST13129136 │         31          │
│                     │ HCAHeartST13146204_HCAHeartST13129137 │         32          │
│                     │ HCAHeartST13146205_HCAHeartST13129138 │         33          │
│                     │ HCAHeartST13146206_HCAHeartST13129139 │         34          │
│                     │ HCAHeartST13146207_HCAHeartST13129140 │         35          │
│                     │ HCAHeartST13146208_HCAHeartST13129141 │         36          │
│                     │ HCAHeartST13168796_HCAHeartST13167740 │         37          │
│                     │ HCAHeartST13180615_HCAHeartST13177112 │         38          │
│                     │ HCAHeartST13180616_HCAHeartST13177113 │         39          │
│                     │ HCAHeartST13180617_HCAHeartST13177114 │         40          │
│                     │ HCAHeartST13180618_HCAHeartST13177115 │         41          │
│                     │ HCAHeartST13180619_H

In [13]:
model.train(
    accelerator='gpu',
    devices=1,
    batch_size=16
)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA RTX 6000 Ada Generation') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1]


Epoch 1/500:   0%|          | 0/500 [00:00<?, ?it/s]

ValueError: Expected parameter loc (Tensor of shape (16, 26)) of distribution Normal(loc: torch.Size([16, 26]), scale: torch.Size([16, 26])) to satisfy the constraint Real(), but found invalid values:
tensor([[nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan,
         nan, nan],
        [nan, nan, nan, nan, nan, nan, nan, nan, nan, 